# Deep RL - DQN with Target Network

### Import required dependencies

In [ ]:
from DeepRL_Support.render import *
import random

import gym
import random
import numpy as np
from collections import namedtuple, deque
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim


import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sample_agent = 'DeepRL_Support/DQNagent_sample.pt'

def set_seed(env, seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    env.reset(seed = seed)

# 2. DQN Algorithm with Target Network

In this notebook we will extend DQN by adding a target network. This revised version of the DQN algorithm is shown below

![DQN_algorithm.png](https://i.postimg.cc/15tMBhjG/DQN-algorithm.png)

### The main changes are:

- We have extended the Agent class' attributes to include 2 DQN networks instead of one (one target and one policy network)

- We have change the method ``get_next_q(.)`` so the q-values are computed using the target network instead of the policy network

- We have added a new method called ``transfer_parameters``(.). This method replaces the parameters of the target network with those of the policy network 

- We have modified the main loop to include instructions that call ``transfer_parameters``(.) after a predefined number of episodes

Compared to DQN, DQN with a target network is more stable and robust, since it updates the network parameters at a lower frequency. It helps with the "catastrophic forgetting" problem that you may have observed in DQN.

## Replay Buffer

We use the same Replay Buffer implementation

**Note**: This implementation of the ReplayMemory class was taken from [***Pytorch DQN tutorial***](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html)

In [ ]:
# This tuple represents one observation in our environment
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward', 'done'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def __len__(self):
        """Returns the length """
        return len(self.memory)

## DQN Network

Let us now define the Multi Layer Perceptron network that will be used as the function approximator for the action-value function (q-function)

In [ ]:
class DQN(nn.Module):
    """DQN Network
        Args:
        input_dim (int): `state` dimension.
        output_dim (int): Number of actions.
        hidden_dim (int): Hidden dimension in fully connected layer
    """
    def __init__(self, num_inputs=8, num_actions=4, hidden_dim_1=32, hidden_dim_2=32):
        super(DQN, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(num_inputs, hidden_dim_1),
            nn.ReLU(),
            nn.Linear(hidden_dim_1, hidden_dim_2),
            nn.ReLU(),
            nn.Linear(hidden_dim_2, num_actions)
        )
        
    def forward(self, x):
        """Returns a Q_value
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, num_inputs)
        Returns:
            torch.Tensor: Q_value, 2-D tensor of shape (n, num_actions)
        """
        return self.layers(x)

## DQN Agent

This class contains the main steps of the Deep Q-learnig algorithm.

Please complete the impletation by filling in two missing parts:
- TODO 1: Add a target network (under the ```__init__``` function)
- TODO 2: Modify `get_next_q` according to the description in "The main changes" at the start of the notebook
- TODO 3: Compute the loss (under the ```optimize``` function)


In [ ]:
class DQNAgent(object):
    """DQN Agent
    This class contains the main steps of the DQN algorithm
    
    Attributes:
    action_value_net (DQN): Function approximator for our action-value function (predictor)
    target_net (DQN): Function approximator for our target action-value function
    loss_fn (MSELoss): Criterion that measures the mean squared error (squared L2 norm) 
                       between each element of the predicted and target q-values.
    optimizer (Adam): Stochastic gradient optimize
    gamma (float): Discount factor
    """
    
    def __init__(self, input_dim=8, output_dim=4, 
                 hidden_dim_1=32, hidden_dim_2=32, gamma=0.99, lr=0.0001):
        """
        Define instance of DQNAgent
        Args:
        input_dim (int): `state` dimension.
        output_dim (int): Number of actions.
        hidden_dim (int): Hidden dimension in fully connected layer
        """
        self.action_value_net = DQN(input_dim, output_dim, hidden_dim_1, hidden_dim_2).to(device)
                
        #TODO 1: Add a target network. Make sure both networks start with same parameters ----------
        self.target_net = None
        #ENDTODO -----------------------------------------------------------------------------------
                        
        self.loss_fn = nn.MSELoss()
        self.optimizer = optim.Adam(self.action_value_net.parameters(), lr=lr)
                
        self.gamma = torch.tensor(gamma).float().to(device)
        
    def obtain_action(self, state, action_space_dim, epsilon):
        """
        Select next action using epsilon-greedy policy
        Args:
        epsilon (float): Threshold used to decide whether a random or maximum-value action 
                         should be taken next
         Returns:
            int: action index
        """        
        with torch.no_grad():
            cur_q = self.action_value_net(torch.from_numpy(state).float().to(device))
        q_value, action = torch.max(cur_q, axis=0)
        action = action if torch.rand(1,).item() > epsilon else torch.randint(0, action_space_dim, (1,)).item()
        action = torch.tensor([action]).to(device)
        return action
    
    def get_next_q(self, state):
        """Returns Q_value for maximum valued action at each state s
        Args:
            x (torch.Tensor): `State` 2-D tensor of shape (n, num_inputs)
        Returns:
            torch.Tensor: Q_value, 1 tensor of shape (n)
        """
        #TODO 2: Compute this function using the correct net =======================================
        
        #ENDTODO ===================================================================================
    
    def optimize(self, batch):
        """Computes `loss` and backpropagation
        Args:
            batch: List[Transition]: Minibatch of `Transition`
        Returns:
            float: loss value
        """
        
        state_batch = torch.stack(batch.state)
        action_batch = torch.stack(batch.action)
        reward_batch = torch.stack(batch.reward)
        next_state_batch = torch.stack(batch.next_state)
                
        # Compute a mask of non-final states and concatenate the batch elements
        # (a final state is the one after which the simulation ends)
        non_final_mask = torch.tensor(tuple(map(lambda s: s.item() is not True,
                                          batch.done)), device=device, dtype=torch.bool)
        non_final_next_states = torch.stack([s for i, s in enumerate(batch.next_state)
                                            if batch.done[i].item() is not True])

        # Compute predicted q-values
        predicted_q = self.action_value_net(state_batch).gather(1, action_batch).reshape(1,-1)
        
        # Compute expected values for non-terminal and terminal states (this is our TD target)
        target_q = torch.zeros(len(batch.state), device=device)
        target_q[non_final_mask] = self.get_next_q(non_final_next_states)
        expected_q = reward_batch.reshape(1,-1)+(self.gamma * target_q)
        
        #TODO 3: Compute loss ----------------------------------------------------------------------
        loss = 0
        #ENDTODO -----------------------------------------------------------------------------------
        
        # Use loss to compute gradient and update policy parameters through backpropagation
        self.optimizer.zero_grad()
        loss.backward(retain_graph=True)
        self.optimizer.step()
                
        return loss.item()
    
    def transfer_parameters(self):
        """Transfer parameters from action-value to target network
        """
        self.target_net.load_state_dict(self.action_value_net.state_dict())
        self.target_net.eval()

### Running Parameters

The parameter ``freq_sync`` defines how often parameters are transferred between networks. Here for every 10 episodes we copy the parameters over to the target network.

Note that here we are controlling the training duration by the number of episodes, instead of using the number of frames as in DQN. This is why the epsilon_decay is dropped.

In [ ]:
# Define running hyper-parameters and epsilon training sequence
# Feel free to change the parameters
memory_capacity = 2500
batch_size = 64
num_episodes = 100
epsilon_start = 1.0
epsilon_end = 0.05
epsilon_decay = 10
gamma = 0.99
lr = 1e-3
hidden_dim_1 = 32
hidden_dim_2 = 32
freq_sync = 10
seed_value = 42

epsilon_by_step = lambda ep_idx: epsilon_end + (epsilon_start - epsilon_end) * np.exp(-1. * ep_idx / epsilon_decay)

# Plotting out epsilon over episodes that should match what you expect
fig, ax = plt.subplots(figsize=(4, 4))
ax.plot([epsilon_by_step(i) for i in range(num_episodes)])
ax.set_xlabel("Num. episodes")
ax.set_ylabel("Epsilon")

### Main Loop and Replay Buffer Control

This is the main loop of our DQN implementation. Here we generate the samples added to the replay memory and train the agent using a batch sampled for the replay memory

In [ ]:
losses_list, rewards_list, episode_len_list = [], [], []
list_epsilon = []
replay_buffer = ReplayMemory(memory_capacity)

env = gym.make("LunarLander-v2")
set_seed(env, seed_value)
n_actions = env.action_space.n
dim_state = env.observation_space.shape[0]

agent = DQNAgent(input_dim=dim_state, 
                 output_dim=n_actions, 
                 hidden_dim_1=hidden_dim_1, 
                 hidden_dim_2=hidden_dim_2, 
                 gamma=gamma, lr=lr)

cur_epsilon = epsilon_start


for i_episode in tqdm(range(num_episodes)):

    state, is_finished, ep_len, losses, rewards = env.reset()[0], False, 0, 0, 0

    cur_epsilon = epsilon_by_step(i_episode+1)
    list_epsilon += [cur_epsilon]
    while not is_finished:
        ep_len += 1
        action = agent.obtain_action(state, n_actions, cur_epsilon)
        next_state, reward, done, truncated, _ = env.step(action.item())
        is_finished = done or truncated
        rewards += reward

        t_s = torch.tensor(state).float().to(device)
        t_r = torch.tensor([reward]).float().to(device)
        t_ns = torch.tensor(next_state).float().to(device)
        t_a = action.to(device)
        t_done = torch.tensor([is_finished]).bool().to(device)

        replay_buffer.push(t_s, t_a, t_ns, t_r, t_done)
        state = next_state

        if len(replay_buffer) > batch_size:
            transitions = replay_buffer.sample(batch_size)
            batch = Transition(*zip(*transitions))
            loss = agent.optimize(batch)
            losses += loss

    losses_list.append(losses / ep_len)
    rewards_list.append(rewards)
    episode_len_list.append(ep_len)

    # Add rule that call transfer_parameters() every freq_sync episodes
    if i_episode % freq_sync == 0:
        agent.transfer_parameters()

    # Every 10 episodes we plot the approximator's progress and performance
    if i_episode % 10 == 0:
        plot_ep(i_episode, rewards_list, losses_list)

In [ ]:
# save the trained network, this is what you will submit along with your ipynb
file_name = 'DeepRL_Exercise.pt' # replace 12345 with your student ID
agent.seed = seed_value
torch.save(agent, file_name)

# Grading

You will be graded based on the performance of your network. Your solution will be executed for a total of 100 trials and the average return will be used to determine your grade. The grading scale is:

| Avg. Return | Marks       |
| ----------- | ----------- |
| < 0         | 1  pt       |
| [0,50)      | 2  pts      |
| [50, 100)   | 3  pts      |
| [100, 150)  | 4  pts      |
| [150, 200)  | 5  pts      |
| â‰¥ 200       | 6  pts      |

Here is the exact grading procedure: 
- We will grade the performance of the `.pt` file, so we will run `DeepRL_Exercise_Grading.ipynb` notebook in the grader (not the `.ipynb` you submitted), obtain `mean_return`, and just compare its value against the table above, feel free to run `DeepRL_Exercise_Grading.ipynb` to test your `.pt` file.
- We will NOT run your `.ipynb`, instead, we will inspect it manually to see if there is some malicious behaviour!

# For your submission:
- Submit both a copy of THIS notebook and the trained network following the naming convention (*DeepRL_Exercise.ipynb* and *DeepRL_Exercise.pt*)
- You can tune the hyper-parameters if needed or modify the architecture of the DQN approximator (don't change the name of the class).
- If you make changes to the architecture if the `DQN` class, please only make changes inside the `nn.Sequential` (if you wish to) because that will get saved in the `.pt` file properly. If you add more things elsewhere in the `DQN` class, there is no guarantee that the grading notebook (`DeepRL_Exercise_Grading.ipynb`) will import `.pt` properly and your network might perform differently than what you expected. 
- Also note that different machines and setup might have different randomness in it too, there is no guarantee that the result you get is what you will get on my end, the only way to make sure that I will get the same number is using Deepnote or Binder for cloud and Docker for local install to check your `.pt` file. Meaning you can train the agent in whatever platform you wish to, but you should check your trained `.pt` in Deepnote or Binder for cloud and Docker for local install